This script implements a simple baseline-CNN model for the Music Genre Classification task.

In [1]:
# System/zip-handling imports
import os, sys
import zipfile

# Imports tensorflow
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, regularizers

# Imports image handling
import cv2
import numpy as np
import skimage

# For generating training and test data
import random

# Save training progress
import csv
from datetime import datetime
from shutil import copyfile  # Making copy of this file instance (including param settings used)

W1212 17:28:48.338946 139798293190400 __init__.py:321] Limited tf.compat.v2.summary API due to missing TensorBoard installation.
W1212 17:28:48.364481 139798293190400 __init__.py:321] Limited tf.compat.v2.summary API due to missing TensorBoard installation.
W1212 17:28:48.383584 139798293190400 __init__.py:352] Limited tf.summary API due to missing TensorBoard installation.


Get a feeling for the nature of the training and evaluation data:

In [2]:
# Get access to zip-archive
archive = zipfile.ZipFile('../data/spectrograms.zip', 'r')
imgdata = archive.read('spectrograms/spectrogram_0000.png')

# Interpret image-data as image
image = cv2.imdecode(np.frombuffer(imgdata, dtype=np.uint8), 1)

# Show image visually (press any key in opening window in order to proceed in code...)
cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Print some data specs
print('Type of image:')
print(type(image))
print('Dimension of a single image file:')
print(image.shape)

Type of image:
<class 'numpy.ndarray'>
Dimension of a single image file:
(128, 128, 3)


Check whether data files are read in in ascending order: otherwise, labels don't match data!!!

In [3]:
# Read in zip-archive
archive = zipfile.ZipFile('../data/spectrograms.zip', 'r')

# For manual checking whether proper string part is retrieved
print(archive.namelist()[1].replace('.', '_').replace('/', '_').split('_'))
print(archive.namelist()[1].replace('.', '_').replace('/', '_').split('_')[-2])
print(int(archive.namelist()[1].replace('.', '_').replace('/', '_').split('_')[-2]))
print('----------------------------------------------')

# Get first number from first file name
num = int(archive.namelist()[1].replace('.', '_').replace('/', '_').split('_')[-2])

# Iterate through file names and check whether they are ordered in ascending order
offset = 2
for i, f_name in enumerate(archive.namelist()[offset:]):
    nxt = int(archive.namelist()[i+offset].replace('.', '_').replace('/', '_').split('_')[-2])
    if nxt <= num:
        print(f_name)
        raise Exception('DATA IS NOT ORDERED IN ASCENDING ORDER! DATA DOESN\'T MATCH LABELS!')
    num = nxt
    
print('No problems detected. Data read in in correct order. Safe to proceed.')

['spectrograms', 'spectrogram', '0000', 'png']
0000
0
----------------------------------------------
No problems detected. Data read in in correct order. Safe to proceed.


Read in training and testing data:

In [ ]:
archive = zipfile.ZipFile('../data/spectrograms.zip', 'r')

#print(archive.namelist())

data_set = []
# Data storage
combined_data = np.empty([1, 128, 128])

# Read in images & store processed instances
for f_name in archive.namelist()[1:]:
    # Get image data from zip file
    zip_img_data = archive.read(f_name)
    image = cv2.imdecode(np.frombuffer(zip_img_data, dtype=np.uint8), 1)
    
    # Normalize image's colors to range [0, 1]
    image = image / 255.0

    #cv2.imshow("Normalized Image", image)
    #cv2.waitKey(0)

    # Grayscale image
    gray_image = skimage.color.rgb2gray(image)

    cv2.imshow("Grayscale Image", gray_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Store grayscaled image
    combined_data = np.append(combined_data, [gray_image], axis=0)
    
# Remove initial, empty datapoint
combined_data = combined_data[1:, :, :]

print('Done reading in... Shape of data array:')
print(combined_data.shape)

Read in labels

In [5]:
# Read in labels
labels_path = '../data/labels.txt'

combined_labels = np.empty([1])

with open(labels_path, 'r') as file:
    #file.readline() # skip the first line
    #labels = [int(line) for line in file]
    for line in file:
        combined_labels = np.append(combined_labels, [int(line)])

# Remove initial, empty datapoint
combined_labels = combined_labels[1:] 
print(combined_labels)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.

In [6]:
## Deprecated. More efficient variant below.
sys.exit()

# Divide data into train and test data
# TODO ##################################################

# Create list of randomly sampled indices
# Then append all data to new train/test arrays wrt to whether it is a test index or not. 
# Then delete old array for storage efficiency!
# ...

# Get set of test-indices which indicates the training data points that have to be reserved for training
percentage_test_data = 0.05
population = range(len(combined_labels))
nr_samples = int(percentage_test_data * len(combined_labels))

test_indices = random.sample(population, nr_samples)
test_indices = sorted(test_indices)

print('Nr. test indices: ' + str(len(test_indices)))
print('Test indices: ' + str(test_indices))


# Split data into training- and test data, respectively
training_data, training_labels = np.empty([1, 128, 128]), np.empty([1])
testing_data, testing_labels = np.empty([1, 128, 128]), np.empty([1])

test_idx_list_idx = 0

##### TODO alternatively: pre-allocate space and just insert.. will resolve & save a lot of struggle!!!

for data_idx in range(len(combined_labels)):

    if test_idx_list_idx < nr_samples and data_idx == test_indices[test_idx_list_idx]:
        test_idx_list_idx += 1
        testing_data = np.append(testing_data, [combined_data[data_idx, :, :]], axis=0)
        testing_labels = np.append(testing_labels, [combined_labels[data_idx]])
        
    else:
        training_data = np.append(training_data, [ combined_data[data_idx, :, :] ], axis=0)
        training_labels = np.append(training_labels, [combined_labels[data_idx]])
        
    
    
print(test_idx_list_idx)

# Remove initial onset (empty) element
training_data, training_labels = training_data[1:,:, :], training_labels[1:]
testing_data, testing_labels = testing_data[1:,:, :], testing_labels[1:]

###### TODO: CORRECT SHAPE???
print('Final:')
print(training_data.shape)
print(training_labels.shape)
print(testing_data.shape)
print(testing_labels.shape)


SystemExit: 

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Divide data into train and test data.

------------------------------------------------------
Training data will be contained in:    training_data
Tetsing  data will be contained in:    testing_data

Training labels will be contained in:  training_labels
Tetsing  labels will be contained in:  testing_labels

In [11]:
# Divide data into train and test data

# Get set of test-indices which indicates the training data points that have to be reserved for training
percentage_test_data = 0.05
population = range(len(combined_labels))
nr_samples = int(percentage_test_data * len(combined_labels))

test_indices = random.sample(population, nr_samples)
test_indices = sorted(test_indices)

print('Nr. test indices: ' + str(len(test_indices)))
print('Test indices: ' + str(test_indices))


# Split data into training- and test data, respectively - Preparation: Create empty arrays to contain data
test_len = len(test_indices)
train_len = len(combined_labels)-len(test_indices)
training_data, training_labels = np.empty([train_len, 128, 128]), np.empty([train_len])
testing_data, testing_labels = np.empty([test_len, 128, 128]), np.empty([test_len])

test_idx_list_idx = 0

# Iterate through all data and assign each data point either to training data or testing data
for data_idx in range(len(combined_labels)):

    if test_idx_list_idx < nr_samples and data_idx == test_indices[test_idx_list_idx]:
        testing_data[test_idx_list_idx, :, :] = combined_data[data_idx, :, :]
        testing_labels[test_idx_list_idx] = combined_labels[data_idx]
        test_idx_list_idx += 1
        
    else:
        i = len(training_labels)-1
        training_data[i, :, :] = combined_data[data_idx, :, :]
        training_labels[i] = combined_labels[data_idx]

training_data = training_data.reshape([len(training_labels), 128, 128, 1])

testing_data = testing_data.reshape([len(testing_labels), 128, 128, 1])
        
print('Final:')
print(training_data.shape)
print(training_labels.shape)
print(testing_data.shape)
print(testing_labels.shape)


Nr. test indices: 50
Test indices: [11, 13, 26, 73, 91, 107, 115, 130, 175, 185, 212, 232, 267, 268, 299, 303, 308, 309, 330, 346, 349, 366, 368, 375, 382, 408, 416, 428, 454, 460, 489, 495, 506, 526, 532, 543, 547, 648, 677, 681, 700, 706, 709, 726, 730, 732, 816, 833, 845, 855]
Final:
(950, 128, 128, 1)
(950,)
(50, 128, 128, 1)
(50,)


Set up the CNN architecture

In [9]:
# Set up model

# Reset tf sessions...
tf.keras.backend.clear_session()  # Destroys the current TF graph and creates a new one.

dimensions = 128
classes = 10

# Set up model architecture in terms of its layers
model = models.Sequential()
model.add(layers.Conv2D(4, (3, 3), activation='relu', input_shape=(dimensions, dimensions, 1),
                        kernel_regularizer=regularizers.l2(1)))
model.add(layers.Dropout(0.1))
#model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(4, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(1)))
model.add(layers.Dropout(0.1))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))
model.add(layers.Flatten())
model.add(layers.Dense(4, activation='relu', kernel_regularizer=regularizers.l2(10.0)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(classes, activation='softmax'))

# Note on regularizer(s), copied from https://www.tensorflow.org/tutorials/keras/overfit_and_underfit:
# l2(0.001) means that every coefficient in the weight matrix of the layer will add 0.001 * weight_coefficient_value**2
# to the total loss of the network.

# Print summary
model.summary()

# Compile model & make some design choices
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.000001,
                                           beta_1=0.9,
                                           beta_2=0.999,
                                           epsilon=1e-07,
                                           amsgrad=False,
                                           name='Adam'
                                           ),
              loss='sparse_categorical_crossentropy',  # Capable of working with regularization
              metrics=['accuracy', 'sparse_categorical_crossentropy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 4)       40        
_________________________________________________________________
dropout (Dropout)            (None, 126, 126, 4)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 124, 124, 4)       148       
_________________________________________________________________
dropout_1 (Dropout)          (None, 124, 124, 4)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 62, 62, 4)         0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 62, 62, 4)         0         
_________________________________________________________________
flatten (Flatten)            (None, 15376)             0

Start training process:
    Run X times 1 training (tensorflow-)epoch and save a model as checkpoint in between. 
    Bit hacky solution, yet, but can be prettyfied.
    
IMPORTANT: 'accuracy'     == accuracy achieved during training on training data;  * the UN-important measure
           'val_accuracy' == accuracy achieved on TEST data AFTER training epoch; * the important measure

In [10]:
# Set up folder for data gathering during training process
now = datetime.now()
TIME_STAMP = now.strftime("_%Y_%d_%m__%H_%M_%S__%f")
MODEL_ID = 'Model_' + TIME_STAMP + '/'
path = 'TrainingRun/' + MODEL_ID

if not os.path.exists(path):
    os.makedirs(path)

# Set up documentation (csv doc) of training progress
with open(path+'training_progress.csv', 'a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["epoch", "loss", "accuracy", "val_loss", "val_accuracy", "sparse_categorical_crossentropy"])
    file.close()


# Save initial model - For preventing memory leaks
name = 'cnn_model_0_acc_0'
model.save(path+name+'.h5')

# Run training
repetitions = 100   # How many repetitions of given nr of epochs
eps = 10            # tf-Epochs
accuracy = 0
min_acc = 0.85

for i in range(repetitions):
    # Live terminal update
    print('Completed epochs so far: ' + str(i*eps))
    
    # Prevent memory leakage
    tf.keras.backend.clear_session()  # Destroys the current TF graph and creates a new one.
    model = tf.keras.models.load_model(path+name+'.h5')  # Reload model

    # Perform x epochs of training
    history = model.fit(training_data, training_labels,
                        epochs=eps,
                        validation_data=(testing_data, testing_labels))
    
    # Get string representation of performed number of training epochs & new model name
    epoch = str((i+1)*eps)
    name = 'cnn_model_'+epoch+'_acc_'+str(history.history['val_accuracy'])
    
    # Save the entire model as a checkpoint and/or final model to a HDF5 file.
    model.save(path+name+'.h5')
    
    # Record training progress
    with open(path+'training_progress.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([epoch,
                         history.history["loss"][0], 
                         history.history["accuracy"][0], 
                         history.history["val_loss"][0], 
                         history.history["val_accuracy"][0],
                         history.history["sparse_categorical_crossentropy"][0]
                         ])
        file.close()

    if accuracy >= min_acc and history.history['val_accuracy'][0] < accuracy:
        # Drop in accuracy on evaluation data. Overfitting?
        break
    else:
        accuracy = history.history['val_accuracy'][0]

Completed epochs so far: 0
Train on 950 samples, validate on 50 samples
Epoch 1/10


W1212 17:11:27.568966 140343296325376 __init__.py:321] Limited tf.compat.v2.summary API due to missing TensorBoard installation.


768/950 [=======================>......] - ETA: 1s - loss: 87.6490 - accuracy: 1.0000 - sparse_categorical_crossentropy: 2.3023

InvalidArgumentError:  Received a label value of -9223372036854775808 which is outside the valid range of [0, 10).  Label values: 0 0 0 0 0 0 -9223372036854775808 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
	 [[node loss/dense_1_loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at /usr/local/lib/python3.5/dist-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_distributed_function_1930]

Function call stack:
distributed_function
